In [3]:
!pip install -U portalocker>=2.0.0

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [1]:
import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch import nn
import time
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

import torch
import transformer_engine.pytorch as te
from transformer_engine.common import recipe
import numpy as np
# from transformer_engine.pytorch import Float8Tensor, E4M3, tensor_to_scale
from transformer_engine.pytorch.fp8 import get_global_fp8_buffer

torch.manual_seed(0)
np.random.seed(0)

someone called API registrations


In [2]:
tokenizer = get_tokenizer("basic_english")
train_iter = AG_NEWS(split="train")


def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class, bias=False)
#         self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
#         self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
    
def base_model_func(model, text, offsets):
    return model(text, offsets)

def train(dataloader, run_model_func, model):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 1
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        if idx > 10:
            break
        optimizer.zero_grad()
        print("model fwd")
        predicted_label = run_model_func(model, text, offsets)
        loss = criterion(predicted_label, label)
        print("model bwd")
        loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        print("optimizer step")
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader, run_model_func, model):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = run_model_func(model, text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [3]:
class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)

    def forward(self, text, offsets):
        return self.embedding(text, offsets)

In [4]:
class TETextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TETextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = te.Linear(embed_dim, num_class, bias=False, params_dtype=torch.float32, primary_weights_in_fp8=True)

    def forward(self, text, offsets, is_first_microbatch=None):
        embedded = self.embedding(text, offsets)
        out = self.fc(embedded, is_first_microbatch=is_first_microbatch)
        return out

In [5]:
train_iter = AG_NEWS(split="train")
# num_class = len(set([label for (label, text) in train_iter]))
num_class = 16
vocab_size = len(vocab)
emsize = 16
# embedding_layer = EmbeddingLayer(vocab_size, emsize, num_class).to(device)
model = TETextClassificationModel(vocab_size, emsize, num_class).to(device)
fp8_recipe = recipe.DelayedScaling(margin=0, interval=1, fp8_format=recipe.Format.E4M3, reduce_amax=False)

# Hyperparameters
EPOCHS = 1  # epoch
LR = 5  # learning rate
BATCH_SIZE = 16  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

assigning weights in fp8
/perfhome/repos/2023/TransformerEngine/transformer_engine/pytorch/float8_tensor.py:107 [Floa8Tensor Torch Dispatch] func: aten.detach.default
 func inplace: False
 mutable args: [False]
/perfhome/repos/2023/TransformerEngine/transformer_engine/pytorch/float8_tensor.py:108 [False]
/perfhome/repos/2023/TransformerEngine/transformer_engine/pytorch/float8_tensor.py:107 [Floa8Tensor Torch Dispatch] func: aten.detach.default
 func inplace: False
 mutable args: [False]
/perfhome/repos/2023/TransformerEngine/transformer_engine/pytorch/float8_tensor.py:108 [False]


In [6]:
# def te_model_func(model, text, offsets):
#     out = model(text, offsets)
# #     print(model.fc.fp8_meta['scaling_fwd'].scale, model.fc.weight._scale)
# #     a_h_flat = model.fc.fp8_meta['scaling_fwd'].amax_history.cpu().numpy() #[1024,3]
# #     print(np.where(a_h_flat > 0.0), a_h_flat[a_h_flat > 0.0])
        
#     return out

In [7]:
# # model = DummyLinear(in_features, out_features)
# # optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# fp8_recipe = recipe.DelayedScaling(margin=0, interval=1, fp8_format=recipe.Format.E4M3, reduce_amax=False)

# for _ in range(3):
#     inp = torch.randn(BATCH_SIZE, emsize, device="cuda")
#     optimizer.zero_grad()
#     # Enable autocasting for the forward pass
#     with te.fp8_autocast(enabled=True, fp8_recipe=fp8_recipe):
#         out = model(inp, is_first_microbatch=None)
    
#     loss = out.sum()
#     loss.backward()
#     optimizer.step()
#     print(model.fc.fp8_meta['scaling_fwd'].scale)
#     a_h_flat = model.fc.fp8_meta['scaling_fwd'].amax_history.cpu().numpy() #[1024,3]
#     print(np.where(a_h_flat > 0.0), a_h_flat[a_h_flat > 0.0])

#     print("----------------------------------")

In [10]:
# def loss_func(pred, true):
#     exp = torch.exp(pred)
#     log_softmax = torch.log(exp/torch.sum(exp, dim=-1, keepdims=True))
#     m, n = pred.shape
#     offset = torch.arange(0, m*n, step = n).reshape(m,1).cuda()
#     idx = true + offset 
#     return -torch.mean(torch.take(log_softmax, idx))

def loss_func(pred, true):
    pred = torch.log(pred)
    return torch.sum(torch.take(pred, true))

In [11]:
total_acc, total_count = 0, 0
log_interval = 1
start_time = time.time()

dataloader = train_dataloader
for epoch in range(1, EPOCHS + 1):
    for idx, (label, text, offsets) in enumerate(dataloader):
        if idx > 10:
            break
        optimizer.zero_grad()
        print("model fwd")
        with te.fp8_autocast(enabled=True, fp8_recipe=fp8_recipe):
            predicted_label = model(text, offsets, is_first_microbatch=None)
        loss = loss_func(predicted_label, label)
#         loss = predicted_label.sum()
#         loss = torch.sum(predicted_label)
        print("model bwd")
        loss.backward()
    #         torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        print("optimizer step")
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

model fwd
fp8.py:  tensor([0.5503, 0.0000, 0.0000], device='cuda:0')
is_first_microbatch None
model bwd
fp8.py:  tensor([0., 0.], device='cuda:0')
optimizer step
/perfhome/repos/2023/TransformerEngine/transformer_engine/pytorch/float8_tensor.py:107 [Floa8Tensor Torch Dispatch] func: aten.add_.Tensor
 func inplace: True
 mutable args: [False, False, False]
/perfhome/repos/2023/TransformerEngine/transformer_engine/pytorch/float8_tensor.py:108 [False, False, False]
/perfhome/repos/2023/TransformerEngine/transformer_engine/pytorch/float8_tensor.py:161 (array([   0,    0, 1023]), array([0, 1, 0])) [0.6234865  0.05447135 0.5502623 ]
model fwd
fp8.py:  tensor([0.6235, 0.0545, 0.0000], device='cuda:0')
is_first_microbatch None
model bwd
fp8.py:  tensor([0., 0.], device='cuda:0')
optimizer step
/perfhome/repos/2023/TransformerEngine/transformer_engine/pytorch/float8_tensor.py:107 [Floa8Tensor Torch Dispatch] func: aten.add_.Tensor
 func inplace: True
 mutable args: [False, False, False]
/perfho

In [10]:
# for epoch in range(1, EPOCHS + 1):
#     epoch_start_time = time.time()
#     train(train_dataloader, te_model_func, model)
#     accu_val = evaluate(valid_dataloader, te_model_func, model)
#     if total_accu is not None and total_accu > accu_val:
#         scheduler.step()
#     else:
#         total_accu = accu_val
#     print("-" * 59)
#     print(
#         "| end of epoch {:3d} | time: {:5.2f}s | "
#         "valid accuracy {:8.3f} ".format(
#             epoch, time.time() - epoch_start_time, accu_val
#         )
#     )
#     print("-" * 59)

TypeError: 'generator' object is not subscriptable

In [6]:
# print("Checking the results of test dataset.")
# accu_test = evaluate(test_dataloader)
# print("test accuracy {:8.3f}".format(accu_test))

In [7]:
# ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}


# def predict(text, text_pipeline):
#     with torch.no_grad():
#         text = torch.tensor(text_pipeline(text))
#         output = model(text, torch.tensor([0]))
#         return output.argmax(1).item() + 1


# ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
#     enduring the season’s worst weather conditions on Sunday at The \
#     Open on his way to a closing 75 at Royal Portrush, which \
#     considering the wind and the rain was a respectable showing. \
#     Thursday’s first round at the WGC-FedEx St. Jude Invitational \
#     was another story. With temperatures in the mid-80s and hardly any \
#     wind, the Spaniard was 13 strokes better in a flawless round. \
#     Thanks to his best putting performance on the PGA Tour, Rahm \
#     finished with an 8-under 62 for a three-stroke lead, which \
#     was even more impressive considering he’d never played the \
#     front nine at TPC Southwind."

# model = model.to("cpu")

# print("This is a %s news" % ag_news_label[predict(ex_text_str, text_pipeline)])

In [12]:
from inspect import currentframe, getframeinfo

def get_current_loc():
    cf = currentframe()
    return f"{getframeinfo(cf).filename}:{cf.f_back.f_lineno}" 


print(f"{get_current_loc()}")

/tmp/ipykernel_3163/2641484001.py:8


In [3]:
train_iter = AG_NEWS(split="train")
# num_class = len(set([label for (label, text) in train_iter]))
num_class = 16
vocab_size = len(vocab)
emsize = 16
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

# Hyperparameters
EPOCHS = 1  # epoch
LR = 5  # learning rate
BATCH_SIZE = 64  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)


In [4]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, base_model_func, model)
    accu_val = evaluate(valid_dataloader, base_model_func, model)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

model fwd
model bwd
optimizer step
model fwd
model bwd
optimizer step
| epoch   1 |     1/ 1782 batches | accuracy    0.211
model fwd
model bwd
optimizer step
| epoch   1 |     2/ 1782 batches | accuracy    0.234
model fwd
model bwd
optimizer step
| epoch   1 |     3/ 1782 batches | accuracy    0.359
model fwd
model bwd
optimizer step
| epoch   1 |     4/ 1782 batches | accuracy    0.312
model fwd
model bwd
optimizer step
| epoch   1 |     5/ 1782 batches | accuracy    0.344
model fwd
model bwd
optimizer step
| epoch   1 |     6/ 1782 batches | accuracy    0.328
model fwd
model bwd
optimizer step
| epoch   1 |     7/ 1782 batches | accuracy    0.375
model fwd
model bwd
optimizer step
| epoch   1 |     8/ 1782 batches | accuracy    0.375
model fwd
model bwd
optimizer step
| epoch   1 |     9/ 1782 batches | accuracy    0.266
model fwd
model bwd
optimizer step
| epoch   1 |    10/ 1782 batches | accuracy    0.328
-----------------------------------------------------------
| end of epoch 